In [ ]:
# %pip install "unstructured[pdf]==0.10.19" pillow pydantic lxml matplotlib tiktoken open_clip_torch torch
# %pip install --upgrade "onnxruntime==1.16.3"
# %pip install pypdf

In [8]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/asharafansari/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/asharafansari/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [14]:
from unstructured.partition.pdf import partition_pdf

image_path = "/Users/asharafansari/Desktop/advanced_rag/images"
# Extract images, tables, and chunk text
raw_pdf_elements = partition_pdf(
    filename=path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=image_path
)

# Categorize by type
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
OMP: Warning #96: Cannot form a team with 4 threads, using 1 instead.
OMP: Hint Consider unsetting KM

In [ ]:
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
from langchain_ollama import OllamaEmbeddings

load_dotenv() 
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)
embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434", 
    model="nomic-embed-text:latest"
)

if not pc.has_index(pinecone_index_name):
    pc.create_index(
        name=pinecone_index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    
index = pc.Index(pinecone_index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) 

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
These summaries will be embedded and used to retrieve the raw text or table elements. \
Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Text summary chain

model = ChatOllama( 
    model = "llama3.2:latest",
    base_url = "http://localhost:11434",
)
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# Apply to text
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# Apply to tables
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

def create_multi_vector_retriever(vectorstore, text_summaries, texts, table_summaries, tables):
    
    store = InMemoryStore()
    id_key = "doc_id"

    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key
    )
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    if table_summaries:
        add_documents(retriever, table_summaries, tables)

    return retriever


retriever_multi_vector = create_multi_vector_retriever(
    vector_store,
    text_summaries,
    texts,
    table_summaries,
    tables
)

In [71]:
query = """How many % Wildfire Damages in Structures Burned is there for the year 2019?"""
docs = vector_store.similarity_search(query)
page_contents=''
for doc in docs:
    page_contents+=doc.page_content
page_contents

'Here is a concise summary of the table:\n\n"Wildland Fire Statistics by Year (2019-2022): \n- Structures burned: 963, 17,904, 5,972, 2,717\n- Residences affected: 46%, 54%, 60%, 46%"Summary: Wildfire statistics from 2018-2022, indicating a significant increase in wildfires each year, with most being human-caused (89%), and larger lightning-caused fires burning more acreage (53%)."Acres Burned by Wildfires"\n\n* Acres burned: millions\n* Number of fires: unknown (missing data)Here is a concise summary optimized for retrieval:\n\n**Wildfire Statistics by Region:**\n\n* **Western US:** Larger wildfires burn more acreage, with over 20,000 fires burning approximately 5.8 million acres in 2022.\n* **Eastern US:** More fires occur on non-federal lands, with 85% (1.5 million acres) of burned acreage on private land.\n\nThis summary captures the key findings and regional differences in wildfire statistics, making it easily searchable for specific information.'

In [72]:
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
rag_prompt_text = ChatPromptTemplate.from_template(template)
chain = rag_prompt_text | model | StrOutputParser() 
response = chain.invoke({"context":page_contents,"question":query})


In [73]:
print(response)

For the year 2019, according to the provided context, 46% of structures burned were damaged by wildfires.
